In [240]:
import pandas as pd
import re
import string
import time

from collections import Counter, defaultdict

from needleman_wunsch import needleman_wunsch

from fuzzywuzzy import fuzz

from operator import itemgetter

pd.set_option('display.max_rows', 1000)

In [2]:
df = pd.read_excel('data/dataYiddish/brandeis/Yiddish titles export.xlsx')

In [4]:
df.sample(5)

,title,trans
69,געזאמעלטע שריפטען / ארטור שניצלער ; איבערזעצט ...,Gezamelṭe shrifṭen / Arṭur Shnitsler ; iber...
6860,NaN,Undzer nayer ḳulṭur-ṿiln : ṿos ṿil der Iḳuf?
3973,NaN,Gabri un di froyen : a shpiel in dray aḳṭen un...
1748,צווישן ביידע וועלט-מלחמות : זכרונות וועגן דעם ...,Tsṿishn beyde ṿelṭ-milḥomes̀ : zikhroynes̀ ṿeg...
6639,NaN,"Erets-Yiśroel, 1936 : a zamlung lider fun Heb..."


In [5]:
len(df)

7220

In [6]:
df_ = df[(df.title.notna() & df.trans.notna())]
len(df_)

4810

In [7]:
type(df_.iloc[0]['title'])

str

In [8]:
re.sub('[^\u0590-\u05fe \-\d]+', '', 'די תורה-דער אמת / מאת רודי זאב זילברברג.')

'די תורה-דער אמת  מאת רודי זאב זילברברג'

In [9]:
df_['title_'] = df_.title.apply(lambda x: re.sub('\(.*?\)', '', x))
df_['trans_'] = df_.trans.apply(lambda x: re.sub('\(.*?\)', '', x))
df_['title_'] = df_.title_.apply(lambda x: re.sub('(?<=[.,])(?=[^\s])', ' ', x))
df_['trans_'] = df_.trans_.apply(lambda x: re.sub('(?<=[.,])(?=[^\s])', ' ', x))
df_['title_'] = df_.title_.apply(lambda x: re.sub('[^\u0590-\u05fe \-\d]+', '', x))
df_['title_'] = df_.title_.apply(lambda x: re.sub('״', '', x))
df_['trans_'] = df_.trans_.apply(lambda x: re.sub('[^\w\d \-]+', '', x))
df_['title_'] = df_.title_.apply(lambda x: re.sub(' +', ' ', x))
df_['trans_'] = df_.trans_.apply(lambda x: re.sub(' +', ' ', x))

/home/iz/miniconda3/envs/kohan/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/iz/miniconda3/envs/kohan/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/iz/miniconda3/envs/kohan/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [14]:
'''with open('heb.txt', 'w') as f:
    for string in df_.title_.values:
        f.write(string)
        f.write('\n')'''

In [16]:
'''with open('lat-low.txt', 'w') as f:
    for string in df_.trans_.values:
        f.write(string.lower())
        f.write('\n')'''

In [103]:
sample = df_.iloc[33]

In [104]:
sample.title_

'דער אידיאט ראמאן אין 4 טיילען און 3 בענדער יבערגעזעצט פון ח גוטמאן '

In [105]:
sample.trans_

'Der eidiot ebergesezt fun H Gutman'

In [139]:
pairs = list(zip(df_.title_.values, df_.trans_.values))

In [141]:
letterpairs = [(pair[0][0], pair[1][0].lower()) for pair in pairs]

In [142]:
letterdictz = defaultdict(str)

for pair in Counter(letterpairs).most_common()[::-1]:
    letterdictz[pair[0][0]] = pair[0][1]

In [143]:
letterdictz

defaultdict(str,
            {' ': 't',
             'ו': 'ṿ',
             '8': '8',
             '0': '6',
             '2': '2',
             'ר': 'r',
             '7': '7',
             'ד': 'd',
             'ּ': 'b',
             'ס': 's',
             'ש': 's',
             'פ': 'f',
             'ג': 'g',
             'צ': 't',
             '6': '6',
             'ת': 't',
             'ע': 'e',
             '1': '1',
             'ה': 'h',
             'י': 'y',
             '-': 'b',
             '4': '4',
             'מ': 'm',
             'א': 'i',
             '3': '3',
             'ח': 'ḥ',
             '5': '5',
             'װ': 'ṿ',
             'ט': 'ṭ',
             'ײ': 'y',
             'כ': 'k',
             'ק': 'ḳ',
             'נ': 'n',
             'ז': 'z',
             'ב': 'b',
             'ל': 'l'})

In [16]:
letterdict[' '] = ' '
letterdict['ן'] = 'n'
letterdict['ם'] = 'm'
letterdict['ץ'] = 'ts'
letterdict['ף'] = 'f'
letterdict['ך'] = 'kh'

In [192]:
hebstr = 'בגדהוזטמנלספרתםןף'
latstr = 'fnmsrfslnmṭzuhdgb'

letterdict = defaultdict(str)

for k, v in zip(hebstr, latstr[::-1]):
    letterdict[k] = v

letterdict['ח'] = 'ḥ' 
letterdict['כ'] = 'kh'
letterdict['ך'] = 'kh'
letterdict['צ'] = 'ts'
letterdict['ץ'] = 'ts'
letterdict['ש'] = 'sh'
letterdict['-'] = '-'
letterdict['ק'] = 'ḳ'
letterdict['װ'] = 'ṿ'
letterdict[' '] = ' '
letterdict['ע'] = 'e'
letterdict['א'] = 'a'
letterdict['י'] = 'i'
letterdict['ײ'] = 'ey'
letterdict['־'] = '-'

for num in range(10):
    letterdict[str(num)] = str(num)
    
for punct in string.punctuation:
    letterdict[punct] = punct

In [185]:
''.join([letterdict[l] for l in 'שטערן'])

'shṭern'

In [181]:
''.join([letterdict[l] for l in 'קנײטלעכלײגערין'])

'ḳneyṭlekhleygerin'

In [180]:
def align(heb, lat, gap=-1, match=2, mismatch=-1):
    alignments = []
    pseudolat = ''.join([letterdict[l] for l in heb])
    print('\n', pseudolat, '\n')
    lat_ = ''.join([l for l in lat if not l in 'ou'])
    indices = needleman_wunsch(pseudolat.lower().split(), lat_.lower().split(), gap=gap, match=match)
    for idx in indices:
        if not None in idx:
            alignment = (heb.split()[idx[0]], lat.split()[idx[1]])
            print(alignment)
            alignments.append(alignment)
    return alignments

In [193]:
example = align(sample.title_, sample.trans_, gap=-2)


 der aidiaṭ raman ain 4 ṭiilen aun 3 bender ibergezetsṭ fun ḥ guṭman  

('3', 'Der')
('בענדער', 'eidiot')
('יבערגעזעצט', 'ebergesezt')
('פון', 'fun')
('ח', 'H')
('גוטמאן', 'Gutman')


In [214]:
h = 'ווילדע רויזען : אונשולדיגע איינפאלען / פון משה נאדיר. '.split()
l = 'Ṿilde royzen : unshuldige aynfalen : tsaykhnung un ṿinyeṭen fun Likhṭenshṭayn...'.split()

for i, word in enumerate(h):
    pseudotrans = ''.join([letterdict[letter] for letter in word]) 
    ratios = []
    for j, trans in enumerate(l):
        ratio = fuzz.ratio(pseudotrans.lower(), trans.lower()) - abs(i-j)
        ratios.append((trans, pseudotrans, ratio))
    maxratio = max(ratios, key=itemgetter(2))
    if maxratio[2] >= 50:
        print({word: maxratio[0]})

{'ווילדע': 'Ṿilde'}
{'רויזען': 'royzen'}
{':': ':'}
{'אונשולדיגע': 'unshuldige'}
{'איינפאלען': 'aynfalen'}
{'פון': 'fun'}


In [225]:
h = sample.title_.split()
l = sample.trans_.split()

for i, word in enumerate(h):
    pseudotrans = ''.join([letterdict[letter] for letter in word]) 
    ratios = []
    for j, trans in enumerate(l):
        ratio = fuzz.ratio(pseudotrans.lower(), trans.lower()) - abs(i-j)
        ratios.append((trans, pseudotrans, ratio))
    maxratio = max(ratios, key=itemgetter(2))
    if maxratio[2] >= 60:
        print({word: maxratio[0]})

{'דער': 'Der'}
{'און': 'fun'}
{'יבערגעזעצט': 'ebergesezt'}
{'פון': 'fun'}
{'גוטמאן': 'Gutman'}


In [ ]:
wordlist = []
worddict = {'אין': 'in', 'און': 'un'}

for heb, lat in zip(df_.title_.values, df_.trans_.values):
    for i, word in enumerate(heb.split()):
        if word in worddict.keys():
            pseudotrans = worddict[word]
        else:
            pseudotrans = ''.join([letterdict[letter] for letter in word]) 
        ratios = []
        for j, trans in enumerate(lat.split()):
            ratio = fuzz.ratio(pseudotrans.lower(), trans.lower()) - abs(i-j)
            ratios.append((trans, pseudotrans, ratio))
        maxratio = max(ratios, key=itemgetter(2))
        if maxratio[2] >= 50:
            print((word, maxratio[0]))
            wordlist.append((word, maxratio[0]))
    print('\n')

In [236]:
wordlist = []
worddict = {'אין': 'in', 'און': 'un'}

for heb, lat in zip(df_.title_.values, df_.trans_.values):
    matches = defaultdict(lambda: ('', 0),)
    for i, word in enumerate(heb.split()):
        if word in worddict.keys():
            pseudotrans = worddict[word]
        else:
            pseudotrans = ''.join([letterdict[letter] for letter in word]) 
        ratios = []
        for j, trans in enumerate(lat.split()):
            ratio = fuzz.ratio(pseudotrans.lower(), trans.lower()) - abs(i-j)
            ratios.append((trans, pseudotrans, ratio))
        maxratio = max(ratios, key=itemgetter(2))
        print(maxratio)
        if maxratio[2] >= 50:
            if matches[maxratio[0]][1] >= maxratio[2]:
                pass
            elif matches[maxratio[0]][1] < maxratio[2]:
                matches[maxratio[0]] = (word, maxratio[2])
    for key, val in matches.items():
        wordlist.append((val[0], key))
        print(val[0], key)
    print()

('Albaṭros', 'albaṭras', 88)
אלבאטראס Albaṭros

('Ale', 'ale', 100)
('ṿerḳ', 'uuerḳ', 67)
('fun', 'fun', 100)
('Daṿid', 'dud', 50)
('Frishman', 'frishman', 100)
אלע Ale
ווערק ṿerḳ
פון fun
דוד Daṿid
פרישמאן Frishman

('Alon', 'elun', 50)
('ha-moreh', 'hmurh', 62)
('Keli', 'khli', 75)
('mivṭa', 'mbṭa', 67)
('shel', 'shl', 86)
('more', 'muri', 50)
('shel', 'snues', 42)
('ʻha-Shomer', 'hshumr', 62)
('ha-Tsaʻirʾ', 'htseir', 62)
('ṿe-ehudeha', 'uauhdih', 35)
עלון Alon
המורה ha-moreh
כלי Keli
מבטא mivṭa
של shel
מורי more
השומר ʻha-Shomer
הצעיר ha-Tsaʻirʾ

('Amol', 'amal', 75)
('in', 'in', 100)
('a', 'a', 100)
('halbn', 'halbn', 100)
('in', 'iubl', 30)
('Yosef', 'iusf', 44)
('Mendelson', 'mendelsan', 89)
אמאל Amol
אין in
א a
האלבן halbn
מענדעלסאן Mendelson

('Amol', 'amal', 75)
('in', 'in', 100)
('a', 'a', 100)
('in', 'iubl', 31)
('zamlbukh', 'zamlbukh', 100)
('far', 'far', 100)
('beleṭrisṭiḳ', 'beleṭrisṭiḳ', 100)
אמאל Amol
אין in
א a
זאמלבוך zamlbukh
פאר far
בעלעטריסטיק beleṭrisṭiḳ

('Di', 'd

('ertsihung', 'gezelshafṭlikh', 34)
('azoy', 'a', 31)
('Idishe', 'grindlikhe', 47)
('Unṭerman', 'ferenṭferung', 49)
('fun', 'fun', 96)
('ṿi', 'di', 36)
('Unṭerman', 'shuuereaun', 52)
('ertsihen', 'ḳamflitsirṭe', 27)
('fragn', 'fragen', 80)
('iber', 'aiber', 78)
('ertsihung', 'ertsihung', 89)
('Bar-Ḥayim', 'bii', 25)
('ḳinder', 'aiden', 39)
('fun', 'fun', 87)
('Yitsḥoḳ', 'itsḥḳ', 70)
('Unṭerman', 'aunṭerman', 81)
אידישע Idishe
און un
אלגעמיינע algemeyne
ערציהונג ertsihung
אזוי azoy
צו tsu
ערציהען ertsihen
אונזערע unzere
קינדער ḳinder
פון fun
אונטערמאן Unṭerman
פראגען fragn
איבער iber
יצחק Yitsḥoḳ

('Idisher-almanakh', 'aidisher-almanakh', 97)
('redagirṭ', 'redagirṭ', 100)
('durkh', 'durkh', 100)
('Sh', 'sh', 100)
('Goreliḳ', 'gareliḳ', 86)
אידישער־אלמאנאך Idisher-almanakh
רעדאגירט redagirṭ
דורך durkh
ש Sh
גארעליק Goreliḳ

('Idisher', 'aidisher', 93)
('arbayṭer', 'arbiiṭer', 75)
('yohr-bukh', 'iahr-bukh', 78)
('un', 'un', 100)
('almanakh', 'almanakh', 100)
אידישער Idisher
ארבייטער arbayṭ

('fun', 'fun', 100)
('Program', 'fragram', 71)
('far', 'far', 100)
('Yidish', 'iidish', 83)
('in', 'in', 100)
('der', 'der', 100)
('elemenṭar-shul', 'elemenṭar-shul', 100)
('tsugeshṭelṭ', 'tsugeshṭelṭ', 100)
('tsum', 'tsum', 100)
('elfṭn', 'elfṭn', 100)
('tsuzamenfor', 'tsuzamenfar', 91)
('fun', 'fun', 100)
('di', 'di', 100)
('Arbeṭer', 'arbeṭer', 100)
('Ring', 'ring', 100)
('shuln', 'shuln', 100)
('Yudl', 'iudl', 75)
('Marḳ', 'marḳ', 100)
('al', 'aa', 49)
פראיעקט Proyeḳṭ
פון fun
פראגראם Program
פאר far
יידיש Yidish
אין in
דער der
עלעמענטאר־שול elemenṭar-shul
צוגעשטעלט tsugeshṭelṭ
צום tsum
עלפטן elfṭn
צוזאמענפאר tsuzamenfor
די di
ארבעטער Arbeṭer
רינג Ring
שולן shuln
יודל Yudl
מארק Marḳ

('Di', 'di', 100)
('pusṭe', 'fusṭe', 80)
('ḳreṭshme', 'ḳreṭshme', 100)
('a', 'a', 100)
('folḳsshṭiḳ', 'falḳsshṭiḳ', 90)
('in', 'in', 100)
('4', '4', 100)
('aḳṭen', 'aḳṭn', 89)
('Perets', 'frts', 60)
('Hirshbeyn', 'hirshbiin', 78)
די Di
פוסטע pusṭe
קרעטשמע ḳreṭshme
א a
פאלקסשטיק folḳsshṭiḳ
אין in
4 4
אקט

('ihre', 'aihre', 89)
('hisṭorishe', 'hisṭarishe', 90)
('faḳṭoren', 'faḳṭaren', 88)
די Di
יודישע yudishe
װירטשאפט ṿirtshafṭ
און un
איהרע ihre
היסטארישע hisṭorishe
פאקטארען faḳṭoren

('Dos', 'das', 67)
('Yudishe', 'iudishe', 86)
('ṿorṭ', 'uuarṭ', 44)
דאס Dos
יודישע Yudishe

('Dos', 'das', 67)
('Yudishe', 'iudishe', 86)
('ṿorṭ', 'ṿarṭ', 75)
('a', 'a', 100)
('liṭerarishe', 'liṭerarishe', 100)
('ḥresṭomaṭye', 'ḥresṭamaṭie', 82)
('tsum', 'tsum', 100)
('lezen', 'lezen', 100)
('in', 'in', 100)
('di', 'di', 100)
('elṭere', 'elṭere', 100)
('grupen', 'grufen', 83)
('fun', 'fun', 100)
('obend-shulen', 'abend-shulen', 92)
('un', 'un', 100)
('in', 'in', 100)
('derheym', 'derheym', 100)
('tsuzamengeshṭelṭ', 'tsuzamengeshṭelṭ', 100)
('durkh', 'durkh', 100)
('O-n', 'a-n', 67)
('redagirṭ', 'redagirṭ', 100)
('durkh', 'durkh', 100)
('a', 'a', 100)
('grupe', 'grufe', 80)
('lehrer', 'lehrer', 100)
('fun', 'fun', 100)
('di', 'di', 100)
('Ṿilner', 'ṿilner', 100)
('obend-shulen', 'abend-shulen', 92)
דאס Dos
י

('traditsyonele', 'ṭraditsianele', 77)
('un', 'un', 100)
('moderne', 'maderne', 86)
('shpayz-geribten', 'shfiiz-geribṭen', 73)
('in', 'in', 100)
('aynglang', 'aiinglang', 82)
('mit', 'miṭ', 67)
('di', 'di', 100)
('idishe', 'aidishe', 92)
('dinim', 'dinim', 100)
('fun', 'fun', 100)
('kashres', 'khshrus', 71)
('-', '-', 100)
('kosher', 'khshr', 73)
('le-Peyseh', 'lfsḥ', 31)
('Bordens', 'bardens', 86)
('farm', 'ferm', 75)
('produkten', 'fraduḳṭen', 56)
('avtaylung', 'abṭiilung', 56)
('fun', 'fun', 100)
('di', 'di', 100)
('Borden', 'barden', 83)
('Kompani', 'ḳamfani', 57)
קאך-בוך Kokh-bukh
פאר far
טראדיציאנעלע traditsyonele
און un
מאדערנע moderne
שפייז־געריבטען shpayz-geribten
אין in
איינגלאנג aynglang
מיט mit
די di
אידישע idishe
דינים dinim
פון fun
כשרות kashres
־ -
כשר kosher
בארדענ׳ס Bordens
פערם farm
פראדוקטען produkten
אבטיילונג avtaylung
בארדען Borden
קאמפאני Kompani

('Der', 'der', 100)
('Ḳonṭinenṭ', 'ḳanṭinenṭ', 89)
('ershṭe', 'ershṭe', 100)
('zamlshrifṭ', 'zamlshrifṭ', 100)
('far'

...

פון fun
שלמה Shlomo
עדעלהייט Edelhayt

('Shmuesn', 'shmuesn', 100)
('ṿegn', 'ṿegn', 100)
('der', 'der', 100)
('Yidisher', 'eydisher', 88)
('liṭeraṭur', 'liṭeraṭur', 100)
('unṭer', 'aunṭer', 91)
('der', 'der', 100)
('Dayṭsher', 'deyṭsher', 88)
('oḳupatsye', 'aḳufatsie', 67)
('Naḥman', 'nḥmn', 80)
('Blumenṭal', 'blumenṭal', 100)
שמועסן Shmuesn
װעגן ṿegn
דער der
ײדישער Yidisher
ליטעראטור liṭeraṭur
אונטער unṭer
דײטשער Dayṭsher
אקופאציע oḳupatsye
נחמן Naḥman
בלומענטאל Blumenṭal

('Shmusen', 'shmuesen', 93)
('ṿegen', 'uuegen', 73)
('bikher', 'bikher', 100)
('Shmuʾel', 'sh', 44)
('Niger', 'niger', 100)
שמועסען Shmusen
וועגען ṿegen
ביכער bikher
ניגער Niger

('Di', 'di', 100)
('Shoḥṭim', 'shuḥṭim', 86)
('shṭime', 'shṭime', 100)
די Di
שוחטים Shoḥṭim
שטימע shṭime

('Shpigl-keśav', 'shfigl-khsb', 61)
('getseylṭe', 'getsiilṭe', 78)
('lider', 'lider', 100)
('Yosef', 'iusf', 44)
('Ḳerler', 'ḳerler', 100)
('Boris', 'baris', 80)
('Ḳarloṿ', 'ḳarlauu', 62)
שפיגל־כתב Shpigl-keśav
געציילטע getseylṭe
לידער 

('yenem', 'ienem', 80)
('un', 'un', 100)
('miṭ', 'miṭ', 100)
('zikh', 'zikh', 100)
('liṭerarishe', 'liṭerarishe', 100)
('eseyen', 'esiien', 67)
('fun', 'fun', 100)
('Yeḥiʾel', 'iḥial', 50)
('Hofer', 'hafer', 80)
מיט Miṭ
יענעם yenem
און un
מיט miṭ
זיך zikh
ליטערארישע liṭerarishe
עסייען eseyen
פון fun
יחיאל Yeḥiʾel
האפער Hofer

('Mosheh', 'mshh', 80)
('Leyb', 'liib', 50)
('Halpern', 'halfern', 86)
משה Mosheh
לייב Leyb
האלפערן Halpern

('Moṭl', 'maṭl', 75)
('editsye', 'fiise', 46)
('dem', 'dem', 100)
('ḥazns', 'ḥzns', 89)
('Shalom-ʻAlekhem', 'shlum-elikhm', 67)
('editsye', 'editsie', 86)
('nokhṿorṭ', 'nakhuuarṭ', 59)
('un', 'un', 100)
('ṭeḳsṭologisher', 'ṭeḳsṭalagisher', 86)
('aparaṭ', 'afaraṭ', 83)
('Ḥana', 'ḥna', 86)
('Shmeruḳ', 'shmeruḳ', 100)
('tsugegegreyṭ', 'tsugegriiṭ', 73)
('tsum', 'tsum', 100)
('druḳ', 'druḳ', 100)
('un', 'un', 100)
('tsunoyfgeshṭelṭ', 'tsunuifgeshṭelṭ', 87)
('dem', 'dem', 100)
('glosar', 'glasar', 83)
('Ṿera', 'uuera', 67)
('Solomon', 'salaman', 57)
מאטל Moṭl
דע

('Tsanger', 'tsanger', 99)
('Binyamin', 'bnimin', 70)
('zayne', 'zab', 43)
('Flaysher', 'fliishr', 66)
פארשפעטיגט Farshpeṭigṭ
די di
שיף shif
שפאנענדע shpanende
געשיכטע geshikhṭe
פון fun
א a
איד Id
משה Mosheh
צאנגער Tsanger
באשריבן bashribn
און un
באארבעט baarbeṭ
דורך durkh
זיינע zayne
אייניקלעך eyniḳlekh
אלימלך Elimelekh
בנימין Binyamin
פליישר Flaysher

('folḳs-lider', 'falḳs-lider', 91)
('fun', 'fun', 100)
('Z', 'z', 97)
('Zeligfeldʾs', 'zeligfelds', 92)
('ḳontserṭ-reperṭuar', 'ḳantserṭ-referṭuar', 86)
פאלקס־לידער folḳs-lider
פון fun
ז Z
זעליגפעלד׳ס Zeligfeldʾs
קאנצערט־רעפערטואר ḳontserṭ-reperṭuar

('Fonen', 'fanen', 80)
('in', 'in', 100)
('bluṭ', 'bluṭ', 100)
('Shpanye', 'shfanie', 71)
('1936', '1936', 100)
('lider', 'lider', 100)
('un', 'un', 100)
('poemen', 'faemen', 67)
('lider', 'ieḳb', 41)
('Glants', 'glants', 100)
פאנען Fonen
אין in
בלוט bluṭ
שפאניע Shpanye
1936 1936
לידער lider
און un
פאעמען poemen
גלאנץ Glants

('Di', 'di', 100)
('fraye', 'friie', 60)
('muze', 'muze', 100)
('

('mit', 'miṭ', 67)
('an', 'au', 50)
('esey', 'esii', 50)
('fun', 'fun', 100)
('Lider', 'ieḳb', 31)
('Fikhman', 'fikhman', 100)
לידער Lider
און un
אידיליעס idilies
שאול Shaʾul
טשערניכאווסקי Ṭsherniḥovsḳi
איריש Idish
שווארץ Shvarts
מיט mit
או an
עסיי esey
פון fun
פיכמאן Fikhman

('Lider', 'lider', 100)
('Lider', 'i', 32)
('Sigal', 'i', 32)
('Sigal', 'sigal', 100)
לידער Lider
סיגאל Sigal

('Der', 'der', 100)
('ligen', 'ligen', 100)
('fun', 'fun', 100)
('der', 'der', 100)
('religyon', 'religeian', 71)
('religyon', 'maḳs', -1)
('der', 'nard', 26)
דער Der
ליגען ligen
פון fun
דער der
רעליגעיאן religyon

('Lomir', 'lamir', 80)
('lakhn--', 'lakhn--', 100)
('zamlung', 'zamlung', 100)
('fun', 'fun', 100)
('mayśehlekh', 'meshhlekh', 74)
('ṿitsn', 'uuitsn', 73)
('un', 'un', 100)
('ṿerṭlekh', 'uuerṭlekh', 82)
('zamlung', 'a', 19)
('Opaṭoṿsḳi', 'afaṭauusḳi', 53)
לאמיר Lomir
לאכן־־ lakhn--
זאמלונג zamlung
פון fun
מעשהלעך mayśehlekh
וויצן ṿitsn
און un
ווערטלעך ṿerṭlekh
אפאטאווסקי Opaṭoṿsḳi

('Lomir', '

('Vakh', 'uuakh', 67)
('iz', 'aiz', 80)
('der', 'der', 100)
('step', 'sṭef', 50)
('dos', 'das', 67)
('ḳol', 'ḳul', 67)
('fun', 'fun', 100)
('der', 'der', 100)
('ṭerḳlṭoyb', 'ṭerḳlṭuib', 78)
וואך Vakh
איז iz
דער der
סטעפ step
דאס dos
קול ḳol
פון fun
טערקלטויב ṭerḳlṭoyb

('ṿakhṭ', 'ṿakhṭ', 98)
('a', 'af', 60)
('ṿakhṭ', 'ṿakhṭ', 100)
('a', 'af', 62)
('dos', 'das', 67)
('Idishe', 'aidishe', 92)
('hoyz', 'hz', 67)
('brenṭ', 'brenṭ', 100)
('a', 'a', 100)
('ṿeygeshray', 'ṿeygeshrey', 90)
('in', 'in', 100)
('der', 'der', 100)
('Idisher', 'aidisher', 93)
('nakhṭ', 'nakhṭ', 100)
('in', 'in', 100)
('Ameriḳe', 'ameriḳe', 100)
('Ḥayim', 'ḥiim', 67)
('Liberman', 'liberman', 100)
װאכט ṿakhṭ
א a
דאס dos
אידישע Idishe
הױז hoyz
ברענט brenṭ
װײגעשרײ ṿeygeshray
אין in
דער der
אידישער Idisher
נאכט nakhṭ
אמעריקע Ameriḳe
חיים Ḥayim
ליבערמאן Liberman

('Ṿayse', 'uuiise', 36)
('shṭoṭ', 'shṭaṭ', 80)
('Efrayim', 'afrim', 67)
('Oyerbakh', 'auierbakh', 71)
שטאט shṭoṭ
אפרים Efrayim
אויערבאך Oyerbakh

('Vayṭshepl', '

In [237]:
len(wordlist)

31749

In [238]:
unique_wordlist = list(set(wordlist))
len(unique_wordlist)

11338

In [ ]:
'''def align(heb, lat, gap=-1, match=2, mismatch=-1):
    for word in lat:
        
    pseudolat = ''.join([letterdict[l] for l in heb])
    lat_ = ''.join([l for l in lat if not l in 'ou'])
    indices = needleman_wunsch(pseudolat.lower().split(), lat_.lower().split(), gap=gap, match=match)
    for idx in indices:
        if not None in idx:
            alignment = (heb.split()[idx[0]], lat.split()[idx[1]])
            words.append(alignment)
            print(alignment)'''

In [239]:
pair_df = pd.DataFrame({'yiddish': [tup[0] for tup in unique_wordlist], 'translit': [tup[1] for tup in unique_wordlist]})

In [243]:
pair_df.groupby(['yiddish', 'translit']).size().head(1000)

yiddish                   translit                 
-                         -                            1
--                        --                           1
-1                        X-1                          1
-סאקאלער                  Soḳoler                      1
01                        10                           1
0291-9191                 1919-1920                    1
02־טן                     20-ṭn                        1
06                        60                           1
1                         1                            1
10                        10                           1
100                       100                          1
10־יאריקן                 10-yoriḳn                    1
10סטן                     20stn                        1
11                        11                           1
11טן                      11ṭn                         1
12                        12                           1
13                        13        

In [244]:
pair_df['source'] = 'brandeis'

In [245]:
pair_df.drop_duplicates().to_csv('data/processed-parallel/brandeis.csv', index=False)